### Setup

apt install requirements

In [2]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


Clone repo and install dependencies

In [3]:
%cd /content

## TODO: use when repo is public
# !git clone https://github.com/jannerm/diffuser.git
## backup for private repo
!curl -L -o diffuser.zip https://www.dropbox.com/s/fe1scd8m2jf7zvj/diffuser.tar?dl=1
!tar -xf diffuser.zip
## /TODO

%cd diffuser
%pip install -f https://download.pytorch.org/whl/torch_stable.html \
                typed-argument-parser \
                scikit-image==0.17.2 \
                scikit-video==1.1.11 \
                gitpython \
                einops \
                pillow \
                free-mujoco-py \
                gym \
                git+https://github.com/JannerM/d4rl.git@0e84a4d29db3ae501043215ce1d91843929f1949 \
                git+https://github.com/aravindr93/mjrl

%pip install -e .

[Errno 2] No such file or directory: '/content'
/home/tomq/Documents/DiffuserMaze2d/diffuser/scripts
/bin/bash: line 1: curl: command not found
tar: diffuser.zip: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
[Errno 2] No such file or directory: 'diffuser'
/home/tomq/Documents/DiffuserMaze2d/diffuser/scripts
Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Cloning https://github.com/JannerM/d4rl.git (to revision 0e84a4d29db3ae501043215ce1d91843929f1949) to /tmp/pip-req-build-lye0zuzz
  Running command git clone --filter=blob:none --quiet https://github.com/JannerM/d4rl.git /tmp/pip-req-build-lye0zuzz
  Running command git rev-parse -q --verify 'sha^0e84a4d29db3ae501043215ce1d91843929f1949'
  Running command git fetch -q https://github.com/JannerM/d4rl.git 0e84a4d29db3ae501043215ce1d91843929f1949
  Running command git checkout -q 0e84a4d29db3ae501043215ce1d91843929f1949
  Resolved https://github.com/JannerM/d4rl.git to commit 0e8

In [4]:
## cythonize mujoco-py at first import
import diffuser.utils as utils

# import importlib
# importlib.reload(utils.colab)

/home/tomq/anaconda3/envs/diff_test/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Compiling /home/tomq/anaconda3/envs/diff_test/lib/python3.8/site-packages/mujoco_py/cymj.pyx because it changed.
[1/1] Cythonizing /home/tomq/anaconda3/envs/diff_test/lib/python3.8/site-packages/mujoco_py/cymj.pyx
running build_ext
building 'mujoco_py.cymj' extension
creating /home/tomq/anaconda3/envs/diff_test/lib/python3.8/site-packages/mujoco_py/generated/_pyxbld_2.0.2.13_38_linuxcpuextensionbuilder
creating /home/tomq/anaconda3/envs/diff_test/lib/python3.8/site-packages/mujoco_py/generated/_pyxbld_2.0.2.13_38_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-38
creating /home/tomq/anaconda3/envs/diff_test/lib/python3.8/site-packages/mujoco_py/generated/_pyxbld_2.0.2.13_38_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-38/home
creating /home/tomq/anaconda3/envs/diff_test/lib/python3.8/site-packages/mujoco_py/generated/_pyxbld_2.0.2.13_38_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-38/home/tomq
creating /home/tomq/anaconda3/envs/diff_test/lib/python3.8/site-packages/mujoc

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem .

pybullet build time: Nov 28 2023 23:51:11


### Conditional sampling

In [ ]:
class Args:
  loadpath = 'logs/azure/hopper-medium-expert-v2/diffusion/H128_T100'
  diffusion_epoch = 'latest'
  n_samples = 4
  device = 'cuda:0'
    
args = Args()

Load diffusion model from disk

In [ ]:
%cd /content/diffuser

diffusion_experiment = utils.load_diffusion(
    args.loadpath, epoch=args.diffusion_epoch)

dataset = diffusion_experiment.dataset
renderer = diffusion_experiment.renderer
model = diffusion_experiment.trainer.ema_model

Initialize environment

In [ ]:
env = dataset.env
obs = env.reset()

Run reverse diffusion

In [ ]:
observations = utils.colab.run_diffusion(
    model, dataset, obs, args.n_samples, args.device)
print(observations.shape)

Render the samples

In [ ]:
sample = observations[-1]
utils.colab.show_sample(renderer, sample)

Render the full reverse diffusion process for the first sample.

This renders `horizon * n_diffusion_steps` images, so takes a few minutes. Increasing `substep` will speed up the video generation, but cause it to skip frames.

In [ ]:
utils.colab.show_diffusion(renderer, observations[:,:1], substep=1)